In [ ]:
#1.Analyzing Sales Data
import pandas as pd
import sqlite3

df = pd.read_csv(r'C:\Users\User\Documents\Python\sales_data.csv', skiprows=2)
print()
print(df.head())

conn = sqlite3.connect("sales.db")

df.to_sql("sales", conn, if_exists="replace", index=False)
print()

query = """
SELECT Date, Product, Category, Quantity, Price
FROM sales
"""
result = pd.read_sql(query, conn)

print()
print(result.head())



         Date       Product     Category  Quantity  Price
0  2023-01-01        Laptop  Electronics        10    800
1  2023-01-01       T-Shirt     Clothing         5     20
2  2023-01-02    Smartphone  Electronics         8    400
3  2023-01-02  Coffee Maker         Home        12     50
4  2023-01-03         Jeans     Clothing        15     30


         Date       Product     Category  Quantity  Price
0  2023-01-01        Laptop  Electronics        10    800
1  2023-01-01       T-Shirt     Clothing         5     20
2  2023-01-02    Smartphone  Electronics         8    400
3  2023-01-02  Coffee Maker         Home        12     50
4  2023-01-03         Jeans     Clothing        15     30


In [75]:
query = "SELECT Date, Product, Category, Quantity, Price FROM sales"
df = pd.read_sql_query(query, conn)


In [76]:

df['Total_Sale'] = df['Quantity'] * df['Price']

# 1.1 Group the data by the Category column and calculate the following aggregate statistics for each category:
# Total quantity sold.
# Average price per unit.
# Maximum quantity sold in a single transaction.
category_stats = df.groupby('Category').agg(
    Total_Quantity_Sold=('Quantity', 'sum'),
    Average_Price_Per_Unit=('Price', 'mean'),
    Max_Quantity_Single_Transaction=('Quantity', 'max')
).reset_index()

print(" Category Statistics:")
print(category_stats)

# 1.2. Identify the top-selling product in each category based on the total quantity sold.
top_products = (
    df.groupby(['Category', 'Product'])['Quantity']
    .sum()
    .reset_index()
    .sort_values(['Category', 'Quantity'], ascending=[True, False])
    .drop_duplicates('Category')
)

print("\n Top-Selling Products by Category:")
print(top_products)

# 1.3. Find the date on which the highest total sales (quantity * price) occurred.
daily_sales = df.groupby('Date')['Total_Sale'].sum()
max_sales_date = daily_sales.idxmax()
max_sales_value = daily_sales.max()

print(f"\n Date with Highest Total Sales: {max_sales_date} (${max_sales_value:.2f})")

conn.close()


 Category Statistics:
      Category  Total_Quantity_Sold  Average_Price_Per_Unit  \
0     Clothing                  157               31.176471   
1  Electronics                  183              276.764706   
2         Home                  144               55.000000   

   Max_Quantity_Single_Transaction  
0                               15  
1                               15  
2                               14  

 Top-Selling Products by Category:
       Category          Product  Quantity
8      Clothing            Jeans        15
27  Electronics         Smart TV        15
46         Home  Pressure Cooker        14

 Date with Highest Total Sales: 2023-01-07 ($15150.00)


In [80]:
#2.Examining Customer Orders
import pandas as pd
import sqlite3


df_csv = pd.read_csv(r'C:\Users\User\Documents\Python\customer_orders.csv',  )  

conn = sqlite3.connect('orders.db')
df_csv.to_sql('customer_orders', conn, index=False, if_exists='replace')


100

In [81]:
query = "SELECT OrderID, CustomerID, Product, Quantity, Price FROM customer_orders"
df = pd.read_sql_query(query, conn)


In [86]:
#2.1.Group the data by CustomerID and filter out customers who have made less than 20 orders.
customer_order_counts = df.groupby('CustomerID')['OrderID'].count()
active_customers = customer_order_counts[customer_order_counts >= 20].index
df_active = df[df['CustomerID'].isin(active_customers)]

print(" Customers with 20 or more orders:")
print(df_active['CustomerID'].unique())


 Customers with 20 or more orders:
[101 102 103 104]


In [85]:
#2.2.Identify customers who have ordered products with an average price per unit greater than $120.
avg_price_per_customer = df.groupby('CustomerID')['Price'].mean()
high_spenders = avg_price_per_customer[avg_price_per_customer > 120].index

print("\n Customers with average price > $120:")
print(high_spenders.tolist())



 Customers with average price > $120:
[102, 104]


In [87]:
#2.3.Find the total quantity and total price for each product ordered, and filter out products that have a total quantity less than 5 units.
df['Total_Price'] = df['Quantity'] * df['Price']

product_stats = df.groupby('Product').agg(
    Total_Quantity=('Quantity', 'sum'),
    Total_Revenue=('Total_Price', 'sum')
).reset_index()

filtered_products = product_stats[product_stats['Total_Quantity'] >= 5]

print("\n Products with total quantity ≥ 5:")
print(filtered_products)



 Products with total quantity ≥ 5:
             Product  Total_Quantity  Total_Revenue
5        Cargo Pants               6            180
15       Dress Shirt               5            125
19      Formal Shirt               6            210
30        Smartphone               5           2000
32       Sport Shoes               5            200
35        Sunglasses               5             75
41  Wireless Earbuds               6            720


In [12]:
#3.
import sqlite3
conn =sqlite3.connect("population (1).db")
cursor=conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables=cursor.fetchall()
print(tables)


[('population',), ('sqlite_sequence',)]


In [4]:
import pandas as pd 
df=pd.read_excel("population_salary_analysis.xlsx")


In [37]:
import sqlite3
import pandas as pd

salary_bands = pd.read_excel("population_salary_analysis.xlsx")

conn = sqlite3.connect("population(1).db")
population = pd.read_sql_query("SELECT * FROM sqlite_master ", conn)
conn.close()

def categorize_salary(salary):
    for _, row in salary_bands.iterrows():
        if row['Min Salary'] <= salary <= row['Max Salary']:
            return row['Salary Band']
    return 'Uncategorized'

population['Salary Band'] = population['rootpage'].apply(categorize_salary)

grouped = population.groupby('Salary Band')['rootpage']
summary = pd.DataFrame({
    'Population Count': grouped.count(),
    'Percentage': grouped.count() / len(population) * 100,
    'Average Salary': grouped.mean(),
    'Median Salary': grouped.median()
}).reset_index()

print(population.head())
print(salary_bands.head())

Empty DataFrame
Columns: [type, name, tbl_name, rootpage, sql, Salary Band]
Index: []
             Salary Band  Percentage  Average Salary  Median Salary  \
0          till $200,000         NaN             NaN            NaN   
1    $200,001 - $400,000         NaN             NaN            NaN   
2    $400,001 - $600,000         NaN             NaN            NaN   
3    $600,001 - $800,000         NaN             NaN            NaN   
4  $800,001 - $1,000,000         NaN             NaN            NaN   

   Number of population  
0                   NaN  
1                   NaN  
2                   NaN  
3                   NaN  
4                   NaN  


In [35]:
print(population.head())
print(salary_bands.head())



Empty DataFrame
Columns: [type, name, tbl_name, rootpage, sql, Salary Band]
Index: []
             Salary Band  Percentage  Average Salary  Median Salary  \
0          till $200,000         NaN             NaN            NaN   
1    $200,001 - $400,000         NaN             NaN            NaN   
2    $400,001 - $600,000         NaN             NaN            NaN   
3    $600,001 - $800,000         NaN             NaN            NaN   
4  $800,001 - $1,000,000         NaN             NaN            NaN   

   Number of population  
0                   NaN  
1                   NaN  
2                   NaN  
3                   NaN  
4                   NaN  
